In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("../data/Chocolate-Sales.csv")

df.info()
df.head(10)

<class 'pandas.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Sales Person   3282 non-null   str  
 1   Country        3282 non-null   str  
 2   Product        3282 non-null   str  
 3   Date           3282 non-null   str  
 4   Amount         3282 non-null   str  
 5   Boxes Shipped  3282 non-null   int64
dtypes: int64(1), str(5)
memory usage: 154.0 KB


,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04/01/2022,"$5,320.00",180
1,Van Tuxwell,India,85% Dark Bars,01/08/2022,"$7,896.00",94
2,Gigi Bohling,India,Peanut Butter Cubes,07/07/2022,"$4,501.00",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27/04/2022,"$12,726.00",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24/02/2022,"$13,685.00",184
5,Van Tuxwell,India,Smooth Sliky Salty,06/06/2022,"$5,376.00",38
6,Oby Sorrel,UK,99% Dark & Pure,25/01/2022,"$13,685.00",176
7,Gunar Cockshoot,Australia,After Nines,24/03/2022,"$3,080.00",73
8,Jehu Rudeforth,New Zealand,50% Dark Bites,20/04/2022,"$3,990.00",59
9,Brien Boise,Australia,99% Dark & Pure,04/07/2022,"$2,835.00",102


In [102]:
print(f"--- Countries :\n {df["Country"].unique().tolist()}\n")
print(f"--- Products :\n {df["Product"].unique()}")

--- Countries :
 ['UK', 'India', 'Australia', 'New Zealand', 'USA', 'Canada']

--- Products :
 <StringArray>
[     'Mint Chip Choco',        '85% Dark Bars',  'Peanut Butter Cubes',
   'Smooth Sliky Salty',      '99% Dark & Pure',          'After Nines',
       '50% Dark Bites',         'Orange Choco',              'Eclairs',
        'Drinking Coco',  'Organic Choco Syrup',            'Milk Bars',
  'Spicy Special Slims',     'Fruit & Nut Bars',           'White Choc',
   'Manuka Honey Choco',         'Almond Choco',      'Raspberry Choco',
 'Choco Coated Almonds',  'Baker's Choco Chips', 'Caramel Stuffed Bars',
       '70% Dark Bites']
Length: 22, dtype: str


In [103]:
# Transforming Amount to float : "$5,320.00" -> 5320.00
df["Amount"] = (
    df["Amount"].astype(str)
    .str.replace(r"[\$,]", "", regex=True)   # remove $ and commas
    .str.strip()
)
df["Amount"] = pd.to_numeric(df["Amount"], errors="coerce")
df["Amount"].head(10)

0     5320.0
1     7896.0
2     4501.0
3    12726.0
4    13685.0
5     5376.0
6    13685.0
7     3080.0
8     3990.0
9     2835.0
Name: Amount, dtype: float64

In [104]:
df["Amount_per_box"] = np.where( # To avoid division by 0
    df["Boxes Shipped"] != 0,
    df["Amount"] / df["Boxes Shipped"],
    np.nan
) 
df.drop(columns="Sales Person",inplace=True) # Irrelevent feature

df.head()

,Country,Product,Date,Amount,Boxes Shipped,Amount_per_box
0,UK,Mint Chip Choco,04/01/2022,5320.0,180,29.555556
1,India,85% Dark Bars,01/08/2022,7896.0,94,84.000000
2,India,Peanut Butter Cubes,07/07/2022,4501.0,91,49.461538
3,Australia,Peanut Butter Cubes,27/04/2022,12726.0,342,37.210526
4,UK,Peanut Butter Cubes,24/02/2022,13685.0,184,74.375000
